In [127]:
import numpy as np
import os
import scipy.io as sio
from collections import Counter
import random
import shutil

dataset = 'amazon'
data_root = '../../dropbox/raw_data/%s' % dataset

In [128]:
cnts = []
parties = []
ss = 'all'
node_labels = {}
row_idx = 0
with open(data_root + '/com-%s.%s.cmty.txt' % (dataset, ss), 'r') as f:
    for row in f:
        row = row.split()
        cnts.append(len(row))
        row = [int(r) for r in row]
        parties.append(row)
        for i in row:
            if not i in node_labels:
                node_labels[i] = set()
            node_labels[i].add(row_idx)
        row_idx += 1
c = 0
selected = []
for i in range(len(cnts)):
    if cnts[i] >= 5000:
        c += 1
        selected.append(i)

s = set()
tot = 0
for i in selected:
    for j in parties[i]:
        s.add(j)
    tot += len(parties[i])
print c, len(s), tot

58 282848 746398


In [134]:
row_idx

75149

In [129]:
adj_list = {}
node_map = {}
node_names = []
with open('../../dropbox/raw_data/%s/com-%s.ungraph.txt' % (dataset, dataset), 'r') as f:
    for row in f:
        if row[0] == '#':
            continue
        row = [int(r) for r in row.split()]
        if not row[0] in adj_list:
            adj_list[row[0]] = []
        if not row[1] in adj_list:
            adj_list[row[1]] = []
        if not row[0] in node_map:
            idx = len(node_map)
            node_map[row[0]] = idx
            node_names.append(row[0])
        if not row[1] in node_map:
            idx = len(node_map)
            node_map[row[1]] = idx
            node_names.append(row[1])
        adj_list[row[0]].append(row[1])
        adj_list[row[1]].append(row[0])

In [130]:
output_folder = '../../dropbox/data/%s-%s' % (dataset, ss)
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)
    
with open(output_folder + '/meta.txt', 'w') as f:
    f.write('%d %d\n' % (len(node_map), len(selected)))

with open(output_folder + '/adj_list.txt', 'w') as f:
    for i in range(len(node_map)):
        node = node_names[i]
        f.write('%d' % len(adj_list[node]))
        for j in adj_list[node]:
            f.write(' %d' % node_map[j])
        f.write('\n')

In [131]:
num_nodes = len(node_map)

with open(output_folder + '/label.txt', 'w') as f:
    for i in range(num_nodes):
        for j in range(len(selected)):
            lb = selected[j]
            node = node_names[i]
            if node in node_labels and lb in node_labels[node]:
                f.write('1 ')
            else:
                f.write('0 ')
        if node in s:
            assert len(node_labels[node])
        f.write('\n')
        
labels = np.loadtxt(output_folder + '/label.txt')
c = 0
idxes = []
for i in range(labels.shape[0]):
    if np.sum(labels[i, :]) > 0:
        c += 1
        idxes.append(i)
print c
print np.sum(labels)

282848
746398.0


In [132]:
idxes = [node_map[i] for i in s]
for i in idxes:
    assert len(node_labels[node_names[i]])
random.shuffle(idxes)
t = int(len(idxes) * 0.9)
with open(output_folder + '/test_idx.txt', 'w') as f:
    for i in range(t, len(idxes)):
        f.write('%d\n' % idxes[i])
        
for i in range(9):
    frac = (i + 1) * 0.01
    num_train = int(len(idxes) * frac)
    with open(output_folder + '/train_idx-%.2f.txt' % frac, 'w') as f:
        for i in range(num_train):
            f.write('%d\n' % idxes[i])
            
for i in range(9):
    frac = (i + 1) * 0.1
    num_train = int(len(idxes) * frac)
    with open(output_folder + '/train_idx-%.1f.txt' % frac, 'w') as f:
        for i in range(num_train):
            f.write('%d\n' % idxes[i])            